In [1]:
import sys
import random
sys.path.append("/pod/2/ke-lab/LUOZ/Singularity/iM6A")

In [2]:
from keras.models import load_model
from pkg_resources import resource_filename
import numpy as np
import pandas as pd
from Bio.Seq import Seq
from Bio.SeqUtils import nt_search
import keras.backend as kb

Using TensorFlow backend.


In [3]:
def one_hot_encode(seq):

    map = np.asarray([[0, 0, 0, 0],
                      [1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])

    seq = seq.upper().replace('A', '\x01').replace('C', '\x02')
    seq = seq.replace('G', '\x03').replace('T', '\x04').replace('N', '\x00')

    return map[np.fromstring(seq, np.int8) % 5]

In [4]:
def categorical_crossentropy_2d(y_true, y_pred):
    # Standard categorical cross entropy for sequence outputs

    return - kb.mean(y_true[:, :, 0]*kb.log(y_pred[:, :, 0]+1e-10)
                   + y_true[:, :, 1]*kb.log(y_pred[:, :, 1]+1e-10))

In [5]:
context = 10000

In [6]:
paths = ('Test/mouseRAC10000_c{}.h5'.format(x) for x in range(1, 6))

In [7]:
models = [load_model(resource_filename('m6AAI', x), custom_objects={'categorical_crossentropy_2d': categorical_crossentropy_2d}) for x in paths]

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


### Read data

In [8]:
Data = pd.read_csv("mm10_Fasta_SNM.csv", keep_default_na=False)

In [9]:
Data

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,LastExonStart,LastExonEnd,m6AStart,...,GeneLength,LastExonLength,Pre,PreSequence,LastExonSequence,m6APosition,POS,Up,Mid,Down
0,Gpr39,chr1,+,125676994,125873862,125677336,125872884,125872369,125873862,125872731,...,196868,1493,195375,GGGACCTGGAAGGGAGAGGAGGATCCGCCGCCAGACAGAGTTCTGC...,GACTGATTGTGGTGACGTTGGCCGTGTGTTGGATGCCCAATCAGAT...,362,195737,GACTGATTGTGGTGACGTTGGCCGTGTGTTGGATGCCCAATCAGAT...,GCCCCCTCATCTTCCTAGCCTCCCGGCGCAGTAACTCTTCCTCCAG...,GCTAAGCCCCAGCCCTTGAGTCCTGAGTCACCACAGACTGGCTCAG...
1,Khdrbs2,chr1,+,32172713,32658568,32172917,32657541,32657443,32658568,32657720,...,485855,1125,484730,GGCCGCACGGGCATCCTCTCCAGCTGAGGCCACGGCCGGAGCCTGG...,CACCAGAAGAATGGGCCACAACTCGCTCCAGCCTGAAGGCACCACC...,277,485007,CACCAGAAGAATGGGCCACAACTCGCTCCAGCCTGAAGGCACCACC...,AATTGCTCCCACTATTTCTTGTATTCCCTTATACTGTTAATGTGAC...,CATACTTTATAAACCATTCTGTAGGTTGATATTCTCTTGATTGTTT...
2,Plcl1,chr1,+,55405920,55754285,55406387,55751463,55751280,55754285,55751363,...,348365,3005,345360,GGAGTGGAGGCCGCCCCGCCGCCGCACTTCCTGGGGCCGCCGTGCT...,GGACAAGGAGACCTGCTGAAGAATGCCAAGAATGAAGCCGTGGAGA...,83,345443,GGACAAGGAGACCTGCTGAAGAATGCCAAGAAT,GAAGCCGTGGAGAACATAAAGCAAATCCAGCTGGCCTGCTTGTCTT...,GGAAGCGATAGAGGAGAAAGAAAGTAGTGAGGAGAACGGGAAGCTG...
3,Sh3bp4,chr1,+,89070414,89155068,89137685,89153351,89153126,89155068,89153400,...,84654,1942,82712,agaccacctctgcgcccgcTGGCGGGGGCCGAGCGCTCTGGGCTGT...,GCCATGTGGAAACCTGCCTATGACTTCTTACTCACCTGGAGCCACC...,274,82986,GCCATGTGGAAACCTGCCTATGACTTCTTACTCACCTGGAGCCACC...,AGTGGGATCCCTGCCCTACTGCTCACTGGAGTTGGGACTCTGCCCA...,CTTGGCCCATCATCTCTCCACAGTGGGCTGTGCCATCACCGGGTGG...
4,Cdh20,chr1,+,104768528,104995481,104934096,104994385,104993879,104995481,104994314,...,226953,1602,225351,AGAAAACGCGCGAAGGAGGGGAAGAAGCGGACCGCGCGCTGGTCTT...,TGTTGGTGCTGCTCATCCTGTCCATGAGGCGACATCGCAAACAACC...,435,225786,TGTTGGTGCTGCTCATCCTGTCCATGAGGCGACATCGCAAACAACC...,CCTGCAGTCAGCCACCTCGGACTCAGAGCAGAGCTTTGATTTCCTC...,GGCCCGCGCCCTTGTGGTGACAGAAGCCCGGAGGCAAAGGGGCACC...
5,Cdh7,chr1,+,109983736,110139001,109994179,110138355,110137861,110139001,110138138,...,155265,1140,154125,TTGGACATCTTCGTGGGTTCACTTGGCACAAGGCTAGTTGCTGGTA...,TGCTGATCCTCCTCATTGTCACTATGAGAAGACGAAAAAAAGAGCC...,277,154402,TGCTGATCCTCCTCATTGTCACTATGAGAAGACGAAAAAAAGAGCC...,TTTAAAAACATCCCAGATAATGTCATTTTTAGGGAATTTATTTGGG...,GACTTATGCATTTGAAGGAAATGGCTCAGTAGCTGAATCTCTCAGT...
6,Clasp1,chr1,+,118389057,118609497,118419723,118606994,118606907,118609497,118609181,...,220440,2590,217850,AAAGGAAGTTCCGGCTGGAAGCGCTCCTGACTCCTCTCTAGCTCTT...,ATGAAGCTGCTGAACTTATATATAAAGAGGGCCCAGACTACCAACA...,2274,220124,ATGAAGCTGCTGAACTTATATATAAAGAGGGCCCAGACTACCAACA...,TTCTCTGCCACCTTTATGGAATCTCACACTTTAAAAGACAGCAAAC...,AAACAATCGGCCACCAAGATAGTTCACACGCATGGTCCTTGCGCCC...
7,Thsd7b,chr1,+,129273343,130219278,129430830,130218180,130218098,130219278,130218485,...,945935,1180,944755,gccccgcggctcccggggACGCACCAGGCAGCCAATGGATGCGCAG...,CAAGAAGCCAAAACCACATCAAAGCACACCTCGCCATCAGAAGCCA...,387,945142,CAAGAAGCCAAAACCACATCAAAGCACACCTCGCCATCAGAAGCCA...,CTCTTCCTGTTTGGAAATGAGAGGAAGAGAACCTGAAGGATTTCGC...,CAATCGAGGCATTTACTGTAAATGGCGAGTCTGACACTGTATTGTT...
8,Brinp3,chr1,+,146494759,146902471,146514717,146902117,146901000,146902471,146901878,...,407712,1471,406241,GACAGTAGATTTCCGAGAGGAAAGGAGAAGAGAAGATAGGGATGGA...,AACATCAACCTACTGGCTCACTCGCATCCAGTCTTTTCTCTACTGC...,878,407119,AACATCAACCTACTGGCTCACTCGCATCCAGTCTTTTCTCTACTGC...,CAGCATGCACTTTGACCCTGAAGCCATTCGGGACCTGATTTTGCAG...,AGATAAGAGACCGTGTAAATAAACTCTCCCCACCCGGTCAGCGCCG...
9,Stx6,chr1,+,155158714,155205659,155158892,155202016,155201939,155205659,155205087,...,46945,3720,43225,GGAGGAGGCTGCCGTGGGGGCCGGCTTGGAGTGGGATTCGACCCGC...,ATCGGCGCCAGTGGTGTGCCATAGCCATCCTCTTCGCGGTCCTGGT...,3148,46373,ATCGGCGCCAGTGGTGTGCCATAGCCATCCTCTTCGCGGTCCTGGT...,TTGGCTTCACCTTGTGCCACAAAGTGAATTTGTAACCAAAACAGAG...,TCCCCCTCCCTCTGACTGCATTGCCTCCCTCCCAAATTGGTAGAAA...


In [11]:
Data = Data.sample(n=1500, random_state=1)

In [12]:
Data = Data.reset_index(drop = True)

In [13]:
Data

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,LastExonStart,LastExonEnd,m6AStart,...,GeneLength,LastExonLength,Pre,PreSequence,LastExonSequence,m6APosition,POS,Up,Mid,Down
0,Kcnd2,chr6,+,21215502,21729805,21216298,21727342,21727164,21729805,21727626,...,514303,2641,511662,GGCGGCTGCCAGCTCCCAGGCTCTATAACTGTCACACTGCACCTGA...,CCGATCCAGCTTAAATGCCAAAATGGAAGAGTGTGTTAAACTAAAC...,462,512124,CCGATCCAGCTTAAATGCCAAAATGGAAGAGTGTGTTAAACTAAAC...,TGTGAATGAGCACAATGAAATGCCATTTATTGATGCCTCTTATGAT...,TTGAATGCAAAACAAAAAAAATATGGAAAACATTTTGATAAAAATT...
1,Zfp772,chr7,-,7202121,7209998,7203439,7209795,7202121,7204359,7204154,...,7877,2238,5639,GGACGCTGGGCGGGACTTTCGGCTTTGCGTGCAGGAGGTTCTTTGA...,GTTGTTGGATCAAAGTGGACAGTGAAATGTCACCCTTTGAGCAGAG...,204,5843,GTTGTTGGATCAAAGTGGACAGTGAAATGTCACCCTTTGAGCAGAG...,GGGTCTTCTAAGGCATCAAGTGACTTATAATGTTGGGGATGCCCCC...,GTGACTGTGGCAAAACTTTTACTAGCACATCCCACCTCAATCGAca...
2,Bccip,chr7,+,133709332,133721145,133709357,133720877,133720706,133721145,133720805,...,11813,439,11374,GCGCAGGCGCAGTGTGACCGGCGACATGGCATCTAAGGCTAAGAAG...,AAGGCAATCTTGAAGTTCAGCTACTCGGTGCAGGGGGAGAGCGACA...,99,11473,AAGGCAATCTTGAAGTTCAGCTACTCGGTGCAGGGGGAGAGCGACACTC,GTCTGGGAGGCAGATGGTCCTTCGATGATGTTCCGATGACGCCTTT...,AAAGACCATCTCTCCGTCTAGTCCATTTCCCCCAGACAGTGACAGG...
3,Fgf14,chr14,-,123977906,124677127,123980345,124676717,123977906,123980482,123980080,...,699221,2576,696645,tttttttttctttttctcttttttctttttttttCCCCCCTTCTCT...,TTGCCATGTACCGAGAACCATCCTTGCATGATGTTGGTGAAACAGT...,401,697046,TTGCCATGTACCGAGAACCATCCTTGCATGATGTTGGTGAAACAGT...,CATGGAATGCCCTACCAGATATGGAATGCCTTTTTAATATCTTTTC...,GCCTGCTGACAGCTACCCATAATCCTTTTTGAGTCCTGTTTCAGCG...
4,Sdr39u1,chr14,-,55897285,55900232,55897602,55900205,55897285,55898012,55897876,...,2947,727,2220,GCGCAGGCGTGCTAGCTCCCTGTCGCTATGCGGGTGCTTGTAGGTG...,GGGTTGTGCTGGGCCGTGGAGGTGGCGCCATCAGTCACATGCTTCT...,135,2355,GGGTTGTGCTGGGCCGTGGAGGTGGCGCCATCAGTCACATGCTTCT...,TCGCCAATTCTTCCCCTGGATACACATTGGTGACCTGGCAGGAATT...,CAGCATCTACAACTACCAATGCTGAGTTTGCCCAGGCCTTGGGTGC...
5,Cdc37l1,chr19,+,28990493,29017569,28990537,29016104,29016005,29017569,29016093,...,27076,1564,25512,AGGGCCGTTGGCGCTGGCGGTTGCCGGGCCCGGACCAGTGGGACAT...,AATCCAGATTCTCTTCAGTGTTGTACCCCTGCCCCTCTGTGCAGTG...,88,25600,AATCCAGATTCTCTTCAGTGTTGTACCCCTGCCCCTCT,GTGCAGTGTAGACTCAGTGGTACACAAAGAAGATGATGACCGAATG...,GAAAGGAAGAGCCTGGCTATTTTCTTGACACTTTTATGGGTGCTGC...
6,A4galt,chr15,-,83226725,83251774,83227500,83228580,83226725,83228594,83228136,...,25049,1869,23180,GGCGCTCCTGTCCGCTGGGCTCAAGCCACCGCCCTCCAGGGCCGCA...,ACAAGTCAATCTTTATGGGCATCTCCTGTTCCCATCTGGAGGAGAC...,457,23637,ACAAGTCAATCTTTATGGGCATCTCCTGTTCCCATCTGGAGGAGAC...,CGGAACCTCGGCATCTCTCTTCTGAGCTGTTTTCCTAATGTCTGGA...,CTCGGAGGCACGGCACAGATGGGAGCCCTACCAGTTGCCCGTACTG...
7,Hexim2,chr11,+,103133344,103139876,103134078,103139065,103138186,103139876,103138637,...,6532,1690,4842,GCTGCTACTGCAGCAAGAGATAGGGCAGAGGCGTTGAGACTCTCAT...,ACTTCCGGTGGTCTTCGGAGCCCCCAGATAGCCCATGAGCCTCATG...,451,5293,ACTTCCGGTGGTCTTCGGAGCCCCCAGATAGCCCATGAGCCTCATG...,TAACCTCGATGTGCTTCACGGGCCCTCCCACTCTGGCTCCGGTGGG...,AGAGGGACTTCTCTGAGGCCTATGAGCGGTACCACACTGAGAGCCT...
8,Chchd5,chr2,+,129129699,129134134,129129855,129133298,129133274,129134134,129133944,...,4435,860,3575,CATGAAGGTTGGACCAATGTGGGATGAGTGGCAGGGACGCTCGACG...,GCACAGCCCCTGCCTGCCTCCTGACAACCCCTTGGACTTTGAGAAA...,670,4245,GCACAGCCCCTGCCTGCCTCCTGACAACCCCTTGGACTTTGAGAAA...,agcacatttcttccactgctgaattagaaccagttgagcagtagac...,gagacagaagcagggatcaaaaggtcaaggtcgtcctttgctgcat...
9,Cdh18,chr15,+,23036462,23474418,23173701,23474418,23473927,23474418,23474176,...,437956,491,437465,GGAGCCGGAGCCTGGGCTCTCCGAGGACCGTGAGCAGCAGTGCCCA...,CCATTGTGGTCCTTTTTATCACCTTGAGGCGTGGCAAAAAAGAGCC...,249,437714,CCATTGTGGTCCTTTTTATCACCTTGAGGCGTGGCAAAAAAGAGCC...,AGTGAAGCTCACTCCGAGACATCAGACCTTGTCCACCCTGGAGAGC...,GCGTGCCCCCTTATGACTCTCTTCAGACTTATGCCTACGAGGGCCA...


In [14]:
PreSequence = Data["PreSequence"].tolist()
Up = Data["Up"].tolist()
Mid = Data["Mid"].tolist()
Down = Data["Down"].tolist()

### Saturation Single Nucleotide Mutation

In [15]:
def SingleNucleotideMutation(DNA):
    
    DNA = DNA.upper()
    
    if DNA == "A":
        Mutation = ["T","C","G"]
    if DNA == "T":
        Mutation = ["A","C","G"]
    if DNA == "C":
        Mutation = ["T","A","G"]
    if DNA == "G":
        Mutation = ["T","C","A"]
    return Mutation


In [16]:
from itertools import product
comb = list(product(["A","T","C","G"],repeat=5))
FiverMer = []
for i in range(len(comb)):
    value = "".join(comb[i])
    FiverMer.append(value)

In [17]:
Dict = dict(zip(FiverMer, [0]*1024))
for key,value in Dict.items():
    motif = key
    Dict[key] = {-1:[], 0:[], 1:[]}

#### Downstream sequence

In [18]:
for i in range(len(Data)):
    
    Pre = PreSequence[i]
    Pre = Pre.upper()
    up = Up[i]
    up = up.upper()
    mid = Mid[i]
    mid = mid.upper()
    down = Down[i]
    down =  down.upper()

    for j in range(53,97,1):
        DNA = mid[j:(j+1)]
        origin = mid[0:j] + DNA + mid[(j+1):101]
        ori0 = origin[(j-4):(j+1)]
        ori1 = origin[(j-3):(j+2)]
        ori2 = origin[(j-2):(j+3)]
        ori3 = origin[(j-1):(j+4)]
        ori4 = origin[(j-0):(j+5)]
        
        Mutation = SingleNucleotideMutation(DNA)
        
        # Mutation[0]
        seq = mid[0:j] + Mutation[0] + mid[(j+1):101]        
        input_sequence = Pre + up + seq + down
        x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
        y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
        m6AAI_prob = y[0, :, 1]
        value0 = m6AAI_prob[(Data.loc[i,"POS"])-1]
        Dvalue0 = value0 - Data.loc[i,"Probabilty"]
        
        motif0 = seq[(j-4):(j+1)]
        motif1 = seq[(j-3):(j+2)]
        motif2 = seq[(j-2):(j+3)]
        motif3 = seq[(j-1):(j+4)]
        motif4 = seq[(j-0):(j+5)]
        
        List1 = [ori0, ori1, ori2, ori3, ori4]
        List2 = [motif0, motif1, motif2, motif3, motif4]
        List3 = list(set(List1).intersection(set(List2))) # intersect
        List4 = list(set(List1).union(set(List2))) # union
        List5 = list(set(FiverMer).difference(set(List4))) # difference
        
        List1 = list(set(List1).difference(set(List3)))
        List2 = list(set(List2).difference(set(List3)))
                
        for k in range(len(List1)):
            Dict[List1[k]][-1].append(Dvalue0)
        for k in range(len(List2)):
            Dict[List2[k]][1].append(Dvalue0)
        for k in range(len(List5)):
            Dict[List5[k]][0].append(Dvalue0)
        for k in range(len(List3)):
            Dict[List3[k]][0].append(Dvalue0)

        # Mutation[1]
        seq = mid[0:j] + Mutation[1] + mid[(j+1):101]        
        input_sequence = Pre + up + seq + down
        x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
        y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
        m6AAI_prob = y[0, :, 1]
        value1 = m6AAI_prob[(Data.loc[i,"POS"])-1]
        Dvalue1 = value1 - Data.loc[i,"Probabilty"]

        motif0 = seq[(j-4):(j+1)]
        motif1 = seq[(j-3):(j+2)]
        motif2 = seq[(j-2):(j+3)]
        motif3 = seq[(j-1):(j+4)]
        motif4 = seq[(j-0):(j+5)]
        
        List1 = [ori0, ori1, ori2, ori3, ori4]
        List2 = [motif0, motif1, motif2, motif3, motif4]
        List3 = list(set(List1).intersection(set(List2))) # intersect
        List4 = list(set(List1).union(set(List2))) # union
        List5 = list(set(FiverMer).difference(set(List4))) # difference
        
        List1 = list(set(List1).difference(set(List3)))
        List2 = list(set(List2).difference(set(List3)))
        
        
        for k in range(len(List1)):
            Dict[List1[k]][-1].append(Dvalue1)
        for k in range(len(List2)):
            Dict[List2[k]][1].append(Dvalue1)
        for k in range(len(List5)):
            Dict[List5[k]][0].append(Dvalue1)
        for k in range(len(List3)):
            Dict[List3[k]][0].append(Dvalue1)
                     
        
        
        # Mutation[2]
        seq = mid[0:j] + Mutation[2] + mid[(j+1):101]        
        input_sequence = Pre + up + seq + down
        x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
        y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
        m6AAI_prob = y[0, :, 1]
        value2 = m6AAI_prob[(Data.loc[i,"POS"])-1]
        Dvalue2 = value2 - Data.loc[i,"Probabilty"]
        
        motif0 = seq[(j-4):(j+1)]
        motif1 = seq[(j-3):(j+2)]
        motif2 = seq[(j-2):(j+3)]
        motif3 = seq[(j-1):(j+4)]
        motif4 = seq[(j-0):(j+5)]
        
        List1 = [ori0, ori1, ori2, ori3, ori4]
        List2 = [motif0, motif1, motif2, motif3, motif4]
        List3 = list(set(List1).intersection(set(List2))) # intersect
        List4 = list(set(List1).union(set(List2))) # union
        List5 = list(set(FiverMer).difference(set(List4))) # difference
        
        List1 = list(set(List1).difference(set(List3)))
        List2 = list(set(List2).difference(set(List3)))
        
        
        for k in range(len(List1)):
            Dict[List1[k]][-1].append(Dvalue2)
        for k in range(len(List2)):
            Dict[List2[k]][1].append(Dvalue2)
        for k in range(len(List5)):
            Dict[List5[k]][0].append(Dvalue2)
        for k in range(len(List3)):
            Dict[List3[k]][0].append(Dvalue2)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


In [19]:
for key,value in Dict.items():
    motif = key
    List1 = Dict[motif][-1]
    List2 = Dict[motif][0]
    List3 = Dict[motif][1]
    Score = [-1]*len(List1) + [0]*len(List2) + [1]*len(List3)
    Value = List1 + List2 + List3
    df = pd.DataFrame({"Score":Score, "Value":Value})
    df.to_csv("./Figure2/MouseLastExon/{}.csv".format(motif), index=0)